# SageMaker Benchmark Evaluation - Basic Usage

This notebook demonstrates the basic user-facing flow for creating and managing benchmark evaluation jobs using the BenchmarkEvaluator with Jinja2 template-based pipeline generation.

## Step 1: Discover Available Benchmarks

Discover the benchmark properties and available options:
https://docs.aws.amazon.com/sagemaker/latest/dg/nova-model-evaluation.html

In [2]:
from sagemaker.train.evaluate import get_benchmarks, get_benchmark_properties
from rich.pretty import pprint

# Configure logging to show INFO messages
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(name)s - %(message)s'
)

# Get available benchmarks
Benchmark = get_benchmarks()
pprint(list(Benchmark))

# Print properties for a specific benchmark
pprint(get_benchmark_properties(benchmark=Benchmark.GEN_QA))

[
│   <_Benchmark.MMLU: 'mmlu'>,
│   <_Benchmark.MMLU_PRO: 'mmlu_pro'>,
│   <_Benchmark.BBH: 'bbh'>,
│   <_Benchmark.GPQA: 'gpqa'>,
│   <_Benchmark.MATH: 'math'>,
│   <_Benchmark.STRONG_REJECT: 'strong_reject'>,
│   <_Benchmark.IFEVAL: 'ifeval'>,
│   <_Benchmark.GEN_QA: 'gen_qa'>,
│   <_Benchmark.MMMU: 'mmmu'>,
│   <_Benchmark.LLM_JUDGE: 'llm_judge'>,
│   <_Benchmark.INFERENCE_ONLY: 'inference_only'>
]

{
│   'modality': 'Multi-Modal (image)',
│   'description': 'Custom Dataset Evaluation – Lets you supply your own dataset for benchmarking, comparing model outputs to reference answers with metrics such as ROUGE and BLEU. gen_qa supports image inference for models which have multimodal support.',
│   'metrics': ['all'],
│   'strategy': 'gen_qa',
│   'subtask_available': False,
│   'subtasks': None
}

## Step 2: Create BenchmarkEvaluator

Create a BenchmarkEvaluator instance with the desired benchmark. The evaluator will use Jinja2 templates to render a complete pipeline definition.

**Required Parameters:**
- `benchmark`: Benchmark type from the Benchmark enum
- `base_model`: Model ARN from SageMaker hub content
- `output_s3_location`: S3 location for evaluation outputs
- `mlflow_resource_arn`: MLflow tracking server ARN for experiment tracking

**Optional Template Fields:**
These fields are used for template rendering. If not provided, defaults will be used:
- `model_package_group`: Model package group ARN
- `source_model_package`: Source model package ARN
- `dataset`: S3 URI of evaluation dataset
- `model_artifact`: ARN of model artifact for lineage tracking (auto-inferred from source_model_package)

In [ ]:
from sagemaker.train.evaluate import BenchMarkEvaluator

# Create evaluator with GEN_QA benchmark
# These values match our successfully tested configuration
evaluator = BenchMarkEvaluator(
    benchmark=Benchmark.GEN_QA,
    model="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28",
    s3_output_path="s3://mufi-test-serverless-smtj/eval/",
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",
    dataset="s3://sagemaker-us-west-2-052150106756/studio-users/d20251107t195443/datasets/2025-11-07T19-55-37-609Z/zc_test.jsonl",
    model_package_group="arn:aws:sagemaker:us-west-2:052150106756:model-package-group/example-name-aovqo", # Optional inferred from model if model package
    base_eval_name="gen-qa-eval-demo",
    # Note: sagemaker_session is optional and will be auto-created if not provided
    # Note: region is optional and will be auto deduced using environment variables - SAGEMAKER_REGION, AWS_REGION
)

pprint(evaluator)

[11/29/25 13:39:45] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=314173;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=126855;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/mufi/Library/Application Support/sagemaker/config.yaml


                    INFO     Resolved MLflow resource ARN:                                    ]8;id=480390;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=329695;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#113\113]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

                    INFO     Model package group provided as ARN:                             ]8;id=572070;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=299487;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#145\145]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package-group/exa                      
                             mple-name-aovqo                                                                       

BenchMarkEvaluator(
│   region=None,
│   sagemaker_session=<sagemaker.core.helper.session_helper.Session object at 0x13cd28e60>,
│   model='arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28',
│   base_eval_name='gen-qa-eval-demo',
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   mlflow_resource_arn='arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment',
│   mlflow_experiment_name=None,
│   mlflow_run_name=None,
│   networking=None,
│   kms_key_id=None,
│   model_package_group='arn:aws:sagemaker:us-west-2:052150106756:model-package-group/example-name-aovqo',
│   benchmark=<_Benchmark.GEN_QA: 'gen_qa'>,
│   subtasks=None,
│   dataset='s3://sagemaker-us-west-2-052150106756/studio-users/d20251107t195443/datasets/2025-11-07T19-55-37-609Z/zc_test.jsonl',
│   evaluate_base_model=True
)

In [ ]:
# # [Optional] BASE MODEL EVAL

# from sagemaker.train.evaluate import BenchMarkEvaluator

# # Create evaluator with GEN_QA benchmark
# # These values match our successfully tested configuration
# evaluator = BenchMarkEvaluator(
#     benchmark=Benchmark.GEN_QA,
#     model="meta-textgeneration-llama-3-2-1b-instruct",
#     s3_output_path="s3://mufi-test-serverless-smtj/eval/",
#     mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",
#     dataset="s3://sagemaker-us-west-2-052150106756/studio-users/d20251107t195443/datasets/2025-11-07T19-55-37-609Z/zc_test.jsonl",
#     # model_package_group="arn:aws:sagemaker:us-west-2:052150106756:model-package-group/example-name-aovqo", # Optional inferred from model if model package
#     base_eval_name="gen-qa-eval-demo",
#     # Note: sagemaker_session is optional and will be auto-created if not provided
#     # Note: region is optional and will be auto deduced using environment variables - SAGEMAKER_REGION, AWS_REGION
# )

# pprint(evaluator)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 # Create evaluator with GEN_QA benchmark                                                    │
│   11 # These values match our successfully tested configuration                                  │
│   12 evaluator = BenchMarkEvaluator(                                                             │
│ ❱ 13 │   benchmark=Benchmark.GEN_QA,                                                             │
│   14 │   model="meta-textgeneration-llama-3-2-1b-instruct",                                      │
│   15 │   s3_output_path="s3://mufi-test-serverless-smtj/eval/",                                  │
│   16 │   mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'Benchmark' is not defined

In [ ]:
# # [Optional] Nova testing IAD Prod

# from sagemaker.train.evaluate import BenchMarkEvaluator

# # Create evaluator with GEN_QA benchmark
# # These values match our successfully tested configuration
# evaluator = BenchMarkEvaluator(
#     benchmark=Benchmark.GEN_QA,
#     # model="arn:aws:sagemaker:us-east-1:052150106756:model-package/bgrv-nova-micro-sft-lora/1",
#     model="arn:aws:sagemaker:us-east-1:052150106756:model-package/test-nova-finetuned-models/3",
#     s3_output_path="s3://mufi-test-serverless-iad/eval/",
#     mlflow_resource_arn="arn:aws:sagemaker:us-east-1:052150106756:mlflow-tracking-server/mlflow-prod-server",
#     dataset="s3://sagemaker-us-east-1-052150106756/studio-users/d20251107t195443/datasets/2025-11-07T19-55-37-609Z/zc_test.jsonl",
#     model_package_group="arn:aws:sagemaker:us-east-1:052150106756:model-package-group/test-nova-finetuned-models", # Optional inferred from model if model package
#     base_eval_name="gen-qa-eval-demo",
#     region="us-east-1",
#     # Note: sagemaker_session is optional and will be auto-created if not provided
#     # Note: region is optional and will be auto deduced using environment variables - SAGEMAKER_REGION, AWS_REGION
# )

# pprint(evaluator)

INFO - botocore.credentials - Found credentials in shared credentials file: ~/.aws/credentials
INFO - sagemaker.modules.evaluate.base_evaluator - Model package group provided as ARN: arn:aws:sagemaker:us-east-1:052150106756:model-package-group/test-nova-finetuned-models


BenchMarkEvaluator(
│   region='us-east-1',
│   sagemaker_session=<sagemaker_core.helper.session_helper.Session object at 0x356a03950>,
│   model='arn:aws:sagemaker:us-east-1:052150106756:model-package/test-nova-finetuned-models/3',
│   base_eval_name='gen-qa-eval-demo',
│   s3_output_path='s3://mufi-test-serverless-iad/eval/',
│   mlflow_resource_arn='arn:aws:sagemaker:us-east-1:052150106756:mlflow-tracking-server/mlflow-prod-server',
│   mlflow_experiment_name=None,
│   mlflow_run_name=None,
│   networking=None,
│   kms_key_id=None,
│   model_package_group='arn:aws:sagemaker:us-east-1:052150106756:model-package-group/test-nova-finetuned-models',
│   benchmark=<_Benchmark.GEN_QA: 'gen_qa'>,
│   subtasks=None,
│   dataset='s3://sagemaker-us-east-1-052150106756/studio-users/d20251107t195443/datasets/2025-11-07T19-55-37-609Z/zc_test.jsonl',
│   evaluate_base_model=True
)

### Optionally update the hyperparameters

In [3]:
pprint(evaluator.hyperparameters.to_dict())

# optionally update hyperparameters
# evaluator.hyperparameters.temperature = "0.1"

# optionally get more info on types, limits, defaults.
# evaluator.hyperparameters.get_info()


[11/29/25 13:26:31] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=665742;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=28065;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Fetching evaluation override parameters for hyperparameters ]8;id=668827;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/benchmark_evaluator.py\benchmark_evaluator.py]8;;\:]8;id=344195;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/benchmark_evaluator.py#495\495]8;;\
                             property                                                                              

                    INFO     Fetching hub content metadata for                                  ]8;id=912465;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py\recipe_utils.py]8;;\:]8;id=530916;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py#201\201]8;;\
                             meta-textgeneration-llama-3-2-1b-instruct from SageMakerPublicHub                     

                    WARNING  No region provided. Using default region.                                 ]8;id=483608;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=394176;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/utils/utils.py#340\340]8;;\

                    INFO     Runs on sagemaker us-west-2, region:us-west-2                             ]8;id=127187;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=740445;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/utils/utils.py#354\354]8;;\

                    INFO     Searching for evaluation recipe with Type='Evaluation' and         ]8;id=26417;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py\recipe_utils.py]8;;\:]8;id=309515;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py#221\221]8;;\
                             EvaluationType='DeterministicEvaluation'                                              

                    INFO     Downloading override parameters from                               ]8;id=762738;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py\recipe_utils.py]8;;\:]8;id=1149;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/recipe_utils.py#249\249]8;;\
                             s3://jumpstart-cache-beta-us-west-2/recipes/open-source-eval-meta-                    
                             textgeneration-llama-3-2-1b-instruct-deterministic_override_params                    
                             _sm_jobs_v1.0.19.json                                                                 

{
│   'max_new_tokens': '8192',
│   'temperature': '0',
│   'top_k': '-1',
│   'top_p': '1.0',
│   'aggregation': '',
│   'postprocessing': 'False',
│   'max_model_len': '12000'
}

## Step 3: Run Evaluation

Start a benchmark evaluation job. The system will:
1. Build template context with all required parameters
2. Render the pipeline definition from `DETERMINISTIC_TEMPLATE` using Jinja2
3. Create or update the pipeline with the rendered definition
4. Start the pipeline execution with empty parameters (all values pre-substituted)

**What happens during execution:**
- CreateEvaluationAction: Sets up lineage tracking
- EvaluateBaseModel & EvaluateCustomModel: Run in parallel as serverless training jobs
- AssociateLineage: Links evaluation results to lineage tracking

In [5]:
# Run evaluation with configured parameters
execution = evaluator.evaluate()
pprint(execution)

print(f"\nPipeline Execution ARN: {execution.arn}")
print(f"Initial Status: {execution.status.overall_status}")

[11/29/25 13:40:20] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=39435;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=899931;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Getting or creating artifact for source:                         ]8;id=774478;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=222956;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#597\597]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package/test-fine                      
                             tuned-models-gamma/28                                                                 

                    INFO     Searching for existing artifact for model package:               ]8;id=672788;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=533927;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#459\459]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package/test-fine                      
                             tuned-models-gamma/28                                                                 

                    INFO     Found existing artifact:                                         ]8;id=555230;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=311641;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#468\468]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:artifact/2b64ef9fe915b3                      
                             138877d772ec489bef                                                                    

                    INFO     Using resolved model_package_group ARN:                          ]8;id=350625;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py\base_evaluator.py]8;;\:]8;id=393598;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/base_evaluator.py#414\414]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:model-package-group/exa                      
                             mple-name-aovqo                                                                       

...

[11/29/25 13:40:22] INFO     Successfully updated pipeline:                                        ]8;id=321336;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=381496;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#208\208]8;;\
                             SagemakerEvaluation-BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b2                 
                             9171c42                                                                               

                    INFO     Starting pipeline execution: gen-qa-eval-demo-1764452422              ]8;id=359442;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=958972;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#263\263]8;;\

                    INFO     Pipeline execution started:                                           ]8;id=73999;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=223527;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#274\274]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation                 
                             -BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b29171c42/execution/9                 
                             5qr3e96dblb                                                                           

BenchmarkEvaluationExecution(
│   arn='arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b29171c42/execution/95qr3e96dblb',
│   name='gen-qa-eval-demo',
│   status=PipelineExecutionStatus(overall_status='Executing', step_details=[], failure_reason=None),
│   last_modified_time=datetime.datetime(2025, 11, 29, 13, 40, 22, 284000, tzinfo=tzlocal()),
│   eval_type=<EvalType.BENCHMARK: 'benchmark'>,
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   steps=[]
)


Pipeline Execution ARN: arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b29171c42/execution/95qr3e96dblb
Initial Status: Executing


### Alternative: Override Subtasks at Runtime

For benchmarks with subtask support, you can override subtasks when calling evaluate():

In [ ]:
# Override subtasks at evaluation time
# execution = mmlu_evaluator.evaluate(subtask="abstract_algebra")  # Single subtask
# execution = mmlu_evaluator.evaluate(subtask=["abstract_algebra", "anatomy"])  # Multiple subtasks

## Step 4: Monitor Execution

Check the job status and refresh as needed:

In [5]:
# Refresh status
execution.refresh()

# Display job status with step details
pprint(execution.status)

# Display individual step statuses
if execution.status.step_details:
    print("\nStep Details:")
    for step in execution.status.step_details:
        print(f"  {step.name}: {step.status}")

PipelineExecutionStatus(
│   overall_status='Executing',
│   step_details=[
│   │   StepDetail(
│   │   │   name='EvaluateCustomModel',
│   │   │   status='Executing',
│   │   │   start_time='2025-11-29T13:26:38.084000-08:00',
│   │   │   end_time='<sagemaker.core.utils.utils.Unassigned object at 0x120de0b60>',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   ),
│   │   StepDetail(
│   │   │   name='EvaluateBaseModel',
│   │   │   status='Executing',
│   │   │   start_time='2025-11-29T13:26:38.083000-08:00',
│   │   │   end_time='<sagemaker.core.utils.utils.Unassigned object at 0x120de0b60>',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   ),
│   │   StepDetail(
│   │   │   name='CreateEvaluationAction',
│   │   │   status='Succeeded',
│   │   │   start_time='2025-11-29T13:26:38.083000-08:00',
│   │   │   end_time='2025-11-29T13:26:42.759000-08:00',
│   │   │   display_name=None,
│   │   │   failure_reason=None
│   │   )
│   ],
│   failure_reason=None
)


Step Details:
  EvaluateCustomModel: Executing
  EvaluateBaseModel: Executing
  CreateEvaluationAction: Succeeded


## Step 5: Wait for Completion

Wait for the pipeline to complete. This provides rich progress updates in Jupyter notebooks:

In [9]:
# Wait for job completion with progress updates
# This will show a rich progress display in Jupyter
execution.wait(target_status="Succeeded", poll=5, timeout=3600)

print(f"\nFinal Status: {execution.status.overall_status}")

╭─────────────────────────────────────────── Pipeline Execution Status ───────────────────────────────────────────╮
│  Overall Status        Succeeded                                                                                │
│  Target Status         Succeeded                                                                                │
│  Elapsed Time          0.5s                                                                                     │
│                                                                                                                 │
│ Pipeline Steps                                                                                                  │
│  Step Name                       Status           Duration                                                      │
│  AssociateLineage                Succeeded        3.3s                                                          │
│  EvaluateCustomModel             Succeeded        3714.0s                                                       │
│  EvaluateBaseModel               Succeeded        5366.2s                                                       │
│  CreateEvaluationAction          Succeeded        2.7s                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[11/29/25 16:21:20] INFO     Final Resource Status: Succeeded                                      ]8;id=401306;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=749;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#979\979]8;;\


Final Status: Succeeded


## Step 6: View Results

Display the evaluation results in a formatted table:

Output Structure:

Evaluation results are stored in S3:

```
s3://your-bucket/output/
└── job_name/
    └── output/
        └── output.tar.gz
```

Extract output.tar.gz to reveal:

```
run_name/
├── eval_results/
│   ├── results_[timestamp].json
│   ├── inference_output.jsonl (for gen_qa)
│   └── details/
│       └── model/
│           └── <execution-date-time>/
│               └── details_<task_name>_#_<datetime>.parquet
└── tensorboard_results/
    └── eval/
        └── events.out.tfevents.[timestamp]
```

In [10]:
pprint(execution.s3_output_path)
# Display results in a formatted table
execution.show_results()

's3://mufi-test-serverless-smtj/eval/'

[11/29/25 16:21:25] INFO     S3 bucket: mufi-test-serverless-smtj, prefix: eval           ]8;id=671086;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=908024;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#130\130]8;;\

                    INFO     Extracted training job name:                                  ]8;id=813615;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=57499;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#63\63]8;;\
                             pipelines-95qr3e96dblb-EvaluateCustomModel-F51y8F3Pg7 from                            
                             step: EvaluateCustomModel                                                             

[11/29/25 16:21:26] INFO     Extracted training job name:                                  ]8;id=745707;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=953308;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#63\63]8;;\
                             pipelines-95qr3e96dblb-EvaluateBaseModel-VA9YzcdIVI from                              
                             step: EvaluateBaseModel                                                               

                    INFO     Searching for results_*.json in                              ]8;id=805603;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=739949;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#150\150]8;;\
                             s3://mufi-test-serverless-smtj/eval/pipelines-95qr3e96dblb-E                          
                             valuateCustomModel-F51y8F3Pg7/output/output/                                          

...

                    INFO     Using metrics from 'all' key (standard benchmark format)      ]8;id=431825;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=75452;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#93\93]8;;\

                    INFO     Using metrics from 'all' key (standard benchmark format)      ]8;id=866976;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py\show_results_utils.py]8;;\:]8;id=697222;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/common_utils/show_results_utils.py#93\93]8;;\

                Custom Model Results                
╭────────────────────────────────┬─────────────────╮
│ Metric                         │           Value │
├────────────────────────────────┼─────────────────┤
│ bleu                           │          6.6928 │
│ bleu_stderr                    │          0.7801 │
│ em                             │           1.23% │
│ em_stderr                      │          0.0018 │
│ f1                             │          19.04% │
│ f1_score_quasi                 │          25.25% │
│ f1_score_quasi_stderr          │          0.0049 │
│ f1_stderr                      │          0.0047 │
│ qem                            │           2.16% │
│ qem_stderr                     │          0.0024 │
│ rouge1                         │          25.69% │
│ rouge1_stderr                  │          0.0047 │
│ rouge2                         │          19.09% │
│ rouge2_stderr                  │          0.0047 │
│ rougeL                         │          25.02% │
│ rougeL_stderr                  │          0.0047 │
╰────────────────────────────────┴─────────────────╯

                 Base Model Results                 
╭────────────────────────────────┬─────────────────╮
│ Metric                         │           Value │
├────────────────────────────────┼─────────────────┤
│ bleu                           │          6.6928 │
│ bleu_stderr                    │          0.7803 │
│ em                             │           1.29% │
│ em_stderr                      │          0.0019 │
│ f1                             │          19.09% │
│ f1_score_quasi                 │          25.22% │
│ f1_score_quasi_stderr          │          0.0049 │
│ f1_stderr                      │          0.0047 │
│ qem                            │           2.18% │
│ qem_stderr                     │          0.0024 │
│ rouge1                         │          25.61% │
│ rouge1_stderr                  │          0.0047 │
│ rouge2                         │          19.04% │
│ rouge2_stderr                  │          0.0047 │
│ rougeL                         │          24.95% │
│ rougeL_stderr                  │          0.0047 │
╰────────────────────────────────┴─────────────────╯

╭─────────────────────────────────────────── Result Artifacts Location ───────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  📦 Full evaluation artifacts available at:                                                                     │
│                                                                                                                 │
│  Custom Model:                                                                                                  │
│    s3://mufi-test-serverless-smtj/eval/pipelines-95qr3e96dblb-EvaluateCustomModel-F51y8F3Pg7/output/output/Non  │
│  e/eval_results/                                                                                                │
│                                                                                                                 │
│  Base Model:                                                                                                    │
│    s3://mufi-test-serverless-smtj/eval/pipelines-95qr3e96dblb-EvaluateBaseModel-VA9YzcdIVI/output/output/None/  │
│  eval_results/                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Step 7: Retrieve an Existing Job

You can retrieve and inspect any existing evaluation job:

In [ ]:
from sagemaker.train.evaluate import EvaluationPipelineExecution
from rich.pretty import pprint


# Get an existing job by ARN
# Replace with your actual pipeline execution ARN
existing_arn = "arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b29171c42/execution/inlsexrd7jes"

# base model only example
# existing_arn = "arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-benchmark/execution/gdp9f4dbv2vi"
existing_execution = EvaluationPipelineExecution.get(
    arn=existing_arn,
    region="us-west-2"
)

pprint(existing_execution)
print(f"\nStatus: {existing_execution.status.overall_status}")

existing_execution.show_results()

[11/29/25 13:35:47] INFO     Extracted s3_output_path from training job                            ]8;id=148252;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=588100;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#367\367]8;;\
                             pipelines-inlsexrd7jes-EvaluateCustomModel-NuPrIoRW4Q:                                
                             s3://mufi-test-serverless-smtj/eval/                                                  

BenchmarkEvaluationExecution(
│   arn='arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-BenchmarkEvaluation-c344c91d-6f62-4907-85cc-7e6b29171c42/execution/inlsexrd7jes',
│   name='inlsexrd7jes',
│   status=PipelineExecutionStatus(
│   │   overall_status='Executing',
│   │   step_details=[
│   │   │   StepDetail(
│   │   │   │   name='EvaluateCustomModel',
│   │   │   │   status='Executing',
│   │   │   │   start_time='2025-11-29T13:26:38.084000-08:00',
│   │   │   │   end_time='<sagemaker.core.utils.utils.Unassigned object at 0x120de0b60>',
│   │   │   │   display_name=None,
│   │   │   │   failure_reason=None
│   │   │   ),
│   │   │   StepDetail(
│   │   │   │   name='EvaluateBaseModel',
│   │   │   │   status='Executing',
│   │   │   │   start_time='2025-11-29T13:26:38.083000-08:00',
│   │   │   │   end_time='<sagemaker.core.utils.utils.Unassigned object at 0x120de0b60>',
│   │   │   │   display_name=None,
│   │   │   │   failure_reason=None
│   │   │   ),
│   │   │   StepDetail(
│   │   │   │   name='CreateEvaluationAction',
│   │   │   │   status='Succeeded',
│   │   │   │   start_time='2025-11-29T13:26:38.083000-08:00',
│   │   │   │   end_time='2025-11-29T13:26:42.759000-08:00',
│   │   │   │   display_name=None,
│   │   │   │   failure_reason=None
│   │   │   )
│   │   ],
│   │   failure_reason=None
│   ),
│   last_modified_time=datetime.datetime(2025, 11, 29, 13, 26, 37, 300000, tzinfo=tzlocal()),
│   eval_type=<EvalType.BENCHMARK: 'benchmark'>,
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   steps=[]
)


Status: Executing


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:22                                                                                   │
│                                                                                                  │
│   19 pprint(existing_execution)                                                                  │
│   20 print(f"\nStatus: {existing_execution.status.overall_status}")                              │
│   21                                                                                             │
│ ❱ 22 existing_execution.show_results()                                                           │
│   23                                                                                             │
│                                                                                                  │
│ /Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-core/src/sagemaker/core/telemetry/tele │
│ metry_logging.py:175 in wrapper                                                                  │
│                                                                                                  │
│   172 │   │   │   │   │   "sagemaker_session is not provided or not valid.",                     │
│   173 │   │   │   │   │   func_name,                                                             │
│   174 │   │   │   │   )                                                                          │
│ ❱ 175 │   │   │   │   return func(*args, **kwargs)                                               │
│   176 │   │                                                                                      │
│   177 │   │   return wrapper                                                                     │
│   178                                                                                            │
│                                                                                                  │
│ /Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/exe │
│ cution.py:1223 in show_results                                                                   │
│                                                                                                  │
│   1220 │   │   self.refresh()                                                                    │
│   1221 │   │                                                                                     │
│   1222 │   │   if self.status.overall_status != "Succeeded":                                     │
│ ❱ 1223 │   │   │   raise ValueError(                                                             │
│   1224 │   │   │   │   f"Cannot show results. Execution status is '{self.status.overall_status}  │
│   1225 │   │   │   │   f"Results are only available after successful execution. "                │
│   1226 │   │   │   │   f"Use execution.wait() to wait for completion or check execution.status   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Cannot show results. Execution status is 'Executing'. Results are only available after successful 
execution. Use execution.wait() to wait for completion or check execution.status for details.

In [ ]:
# Run evaluation with configured parameters
execution = evaluator.evaluate()
pprint(execution)

print(f"\nPipeline Execution ARN: {execution.arn}")
print(f"Initial Status: {execution.status.overall_status}")

INFO - sagemaker.modules.evaluate.benchmark_evaluator - Getting or creating artifact for source: arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28
INFO - sagemaker.modules.evaluate.base_evaluator - Searching for existing artifact for model package: arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28
INFO - sagemaker.modules.evaluate.base_evaluator - Found existing artifact: arn:aws:sagemaker:us-west-2:052150106756:artifact/2b64ef9fe915b3138877d772ec489bef
INFO - sagemaker.modules.evaluate.benchmark_evaluator - Resolved model info - base_model_name: meta-textgeneration-llama-3-2-1b-instruct, base_model_arn: arn:aws:sagemaker:us-west-2:aws:hub-content/SageMakerPublicHub/Model/meta-textgeneration-llama-3-2-1b-instruct/1.10.0, source_model_package_arn: arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/28
INFO - sagemaker.modules.evaluate.benchmark_evaluator - Using configured hyperparamet

[11/22/25 12:24:36] INFO     Updating pipeline resource.                                         ]8;id=707103;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=260368;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/resources.py#30485\30485]8;;\

INFO - sagemaker_core.main.resources - Updating pipeline resource.
INFO - sagemaker.modules.evaluate.execution - Successfully updated pipeline: SagemakerEvaluation-benchmark
INFO - sagemaker.modules.evaluate.execution - Starting pipeline execution: gen-qa-eval-demo-1763843077
INFO - sagemaker.modules.evaluate.execution - Pipeline execution started: arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-benchmark/execution/gv93gtwgr7w8


BenchmarkEvaluationExecution(
│   arn='arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-benchmark/execution/gv93gtwgr7w8',
│   name='gen-qa-eval-demo',
│   status=PipelineExecutionStatus(overall_status='Executing', step_details=[], failure_reason=None),
│   last_modified_time=datetime.datetime(2025, 11, 22, 12, 24, 37, 828000, tzinfo=tzlocal()),
│   eval_type=<EvalType.BENCHMARK: 'benchmark'>,
│   s3_output_path='s3://mufi-test-serverless-smtj/eval/',
│   steps=[]
)


Pipeline Execution ARN: arn:aws:sagemaker:us-west-2:052150106756:pipeline/SagemakerEvaluation-benchmark/execution/gv93gtwgr7w8
Initial Status: Executing


## Step 8: List All Benchmark Evaluations

Retrieve all benchmark evaluation executions:

In [ ]:
# Get all benchmark evaluations (returns iterator)
all_executions_iter = BenchMarkEvaluator.get_all(region="us-west-2")
all_executions = list(all_executions_iter)

print(f"Found {len(all_executions)} evaluation(s)\n")
for exec in all_executions[:5]:  # Show first 5
    print(f"  {exec.name}: {exec.status.overall_status}")

[11/29/25 13:41:19] INFO     Extracted s3_output_path from training job                            ]8;id=166943;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=816278;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#367\367]8;;\
                             pipelines-95qr3e96dblb-EvaluateCustomModel-F51y8F3Pg7:                                
                             s3://mufi-test-serverless-smtj/eval/                                                  

                    INFO     Extracted s3_output_path from training job                            ]8;id=521868;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py\execution.py]8;;\:]8;id=351282;file:///Volumes/workplace/sagemaker-python-sdk-staging/sagemaker-train/src/sagemaker/train/evaluate/execution.py#367\367]8;;\
                             pipelines-inlsexrd7jes-EvaluateCustomModel-NuPrIoRW4Q:                                
                             s3://mufi-test-serverless-smtj/eval/                                                  

Found 2 evaluation(s)

  95qr3e96dblb: Executing
  inlsexrd7jes: Executing


## Step 9: Stop a Running Job (Optional)

You can stop a running evaluation if needed:

In [ ]:
# Uncomment to stop the job
# existing_execution.stop()
# print(f"Execution stopped. Status: {execution.status.overall_status}")

/Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/shapes.py:2350: UserWarning: Field name "schema" in "AutoMLSnowflakeDatasetDefinition" shadows an attribute in parent "Base"
  class AutoMLSnowflakeDatasetDefinition(Base):
/Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/shapes.py:6372: UserWarning: Field name "schema" in "SnowflakeDatasetDefinition" shadows an attribute in parent "Base"
  class SnowflakeDatasetDefinition(Base):


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/mufi/Library/Application Support/sagemaker/config.yaml


[11/22/25 18:32:01] WARNING  No boto3 session provided. Creating a new session.                        ]8;id=549422;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=573139;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/utils.py#339\339]8;;\

                    WARNING  No config provided. Using default config.                                 ]8;id=278829;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=978800;file:///Users/mufi/.local/share/mise/installs/python/3.12.12/lib/python3.12/site-packages/sagemaker_core/main/utils.py#347\347]8;;\

Succeeded


AWS service error when stopping pipeline execution: Pipeline execution with ARN arn:aws:sagemaker:us-west-2:052150106756:pipeline/sagemakerevaluation-benchmark/execution/7rr30o7c2qfb status 'Succeeded'. Only pipelines with 'Executing' status can be stopped.


## Understanding the Pipeline Structure

The rendered pipeline definition includes:

**4 Steps:**
1. **CreateEvaluationAction** (Lineage): Sets up tracking
2. **EvaluateBaseModel** (Training): Evaluates base model
3. **EvaluateCustomModel** (Training): Evaluates custom model
4. **AssociateLineage** (Lineage): Links results

**Key Features:**
- Template-based: Uses Jinja2 for flexible pipeline generation
- Parallel execution: Base and custom models evaluated simultaneously
- Serverless: No need to manage compute resources
- MLflow integration: Automatic experiment tracking
- Lineage tracking: Full traceability of evaluation artifacts

**Typical Execution Time:**
- Total: ~10-12 minutes
- Downloading phase: ~5-7 minutes (model and dataset)
- Training phase: ~3-5 minutes (running evaluation)
- Lineage steps: ~2-4 seconds each